In [ ]:
# fastai 0.7.2, kinda worked - downloaded into a folder. But then dunno how to import and use it properly...
#!pip install -e git+https://github.com/fastai/fastai1@e85667cfae2e6873b1bb026195b5d09a74dfcff9#egg=fastai07
#%load_ext autoreload
#%autoreload 2
#%matplotlib inline

In [ ]:
!pip install fastai==0.7.0 --no-deps
!pip install scikit-learn==0.21.3
# fastai depends also on an older version of torch
!pip install torch==0.4.1 torchvision==0.2.1
!pip show fastai

In [ ]:
!pip install treeinterpreter
!pip install waterfallcharts

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/heroes-of-the-storm-matches-sample/export_flat_form.csv')

In [ ]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

### Preprocessing
Check info on the events, i.e. date range, plot date distribution to get a clearer view about the data, then drop the date column entirely since it is useless information for our models.

In [ ]:
df.date = pd.to_datetime(df["date"])
dates_ordinal = pd.Series([x.toordinal() for x in df.date])

First check the dates and filter the ones that would be too old for us to be usable.

In [ ]:
# plot non-parametric kde on numeric datenum
ax = dates_ordinal.plot(kind='kde')
# rename the xticks with labels
x_ticks = ax.get_xticks()
ax.set_xticks(x_ticks[::2])
xlabels = [datetime.datetime.fromordinal(int(x)).strftime('%Y-%m-%d') for x in x_ticks[::2]]
ax.set_xticklabels(xlabels)

let's keep just last 5 months dates

In [ ]:
#see if there are any missing values

In [ ]:
dates_mask = (df.date > '2020-10-01')
df_newer_dates = df.loc[dates_mask]
dates_ordinal_newer = pd.Series([x.toordinal() for x in df_newer_dates.date])

In [ ]:
# plot non-parametric kde on numeric datenum
ax = dates_ordinal_newer.plot(kind='kde')
# rename the xticks with labels
x_ticks = ax.get_xticks()
ax.set_xticks(x_ticks[::2])
xlabels = [datetime.datetime.fromordinal(int(x)).strftime('%Y-%m-%d') for x in x_ticks[::2]]
ax.set_xticklabels(xlabels)

In [ ]:
min_date = df.date.min()
max_date = df.date.max()
print("older match: "+ str(min_date)+"   newest match: "+str(max_date))

In [ ]:
min_date = df_newer_dates.date.min()
max_date = df_newer_dates.date.max()
print("older match: "+ str(min_date)+"   newest match: "+str(max_date))

In [ ]:
sample_size = 30000

In [ ]:
df_raw = df_newer_dates.drop(columns=['date'])
print(df_raw.shape)
df_raw = df_raw.sample(n = sample_size, random_state = 1, axis = 0)
print(df_raw.shape)
df_raw.reset_index(drop=True, inplace=True)
#print(df_raw.tail())

In [ ]:
invert_ratio = 0.5
n_inv = int(len(df_raw) * invert_ratio)
df_standard, df_inverted = split_vals(df_raw, n_inv)
#print(df_inverted.head())
df_inverted = df_inverted.rename(columns={"winnerA": "loserA", "winnerB": "loserB", "winnerC": "loserC", "winnerD": "loserD", "winnerE": "loserE",
                            "loserA": "winnerA", "loserB": "winnerB", "loserC": "winnerC", "loserD": "winnerD", "loserE": "winnerE"})
#df.loc[idx] = df.loc[idx].rename(columns={'R':'L','L':'R'})
df_inverted['outcome'] = 0
df_mixed = df_standard.append(df_inverted)
#print(df_mixed.head())
#print(df_mixed.tail())
df_mixed.outcome.mean()

In [ ]:
train_cats(df_mixed)
print([len(df_mixed[x].cat.categories) for x in df.columns[1:13]])

### if it throws error here, it means tables were cleared, need to re-run from the import of csv:

In [ ]:

hero_names = df_mixed['winnerA'].cat.categories
heroes_total = max(hero_names)
map_names = df_mixed['game_map'].cat.categories

print([len(df_mixed[x].cat.categories) for x in df.columns[1:13]], "  heroes_total =", heroes_total)
print(hero_names[:9], "...")
#df_mixed.dtypes

In [ ]:
#clear up memory:
dates_ordinal_newer = 0
dates_ordinal = 0
df = pd.DataFrame()
df_raw = pd.DataFrame()
df_standard = pd.DataFrame()
df_inverted = pd.DataFrame()

Great, shuffling worked. Another problem is the format of the table itself. We are more interested in relations between features more than anything else, therefore let's transform the table into something mode understandable for the Tree type model. For that, the first idea to try - split the features into "multiple-hot" encoding table. Not sure if this is even a thing, but let's try that. A one-hot encoder would give us a great start, so let's use that.

In [ ]:
winner_cols = ['winnerA', 'winnerB', 'winnerC', 'winnerD', 'winnerE']
loser_cols = ['loserA', 'loserB', 'loserC', 'loserD', 'loserE']
dummies_data = pd.DataFrame(columns = hero_names, dtype = 'int8')

In [ ]:
def hero_hot_transform(game_map = '', game_type = 'UnrankedDraft', winners = [], losers = [], bans = []):
    new_row = pd.Series(index = hero_names, dtype = 'int8')
    for x in hero_names:
        if x in winners:
            new_row[x] = 2
        if x in losers:
            new_row[x] = 1
    game_map_index = df_mixed['game_map'].cat.categories.get_loc(game_map) 
    game_type_index = df_mixed['game_type'].cat.categories.get_loc(game_type)        

    #print(dict( enumerate(df_mixed['game_map'].cat.categories ) ))
    new_row = new_row.append(pd.Series([game_map_index, game_type_index], index = ['game_map','game_type']))
    return new_row

In [ ]:
game_map = 'Garden of Terror'
game_type = 'UnrankedDraft'
winners=['Auriel', 'Muradin']
losers=['Illidian', 'Abathur']
row = [hero_hot_transform(game_map = game_map, game_type=game_type, winners = winners, losers=losers)]
row

In [ ]:
for index in range(0, int(len(df_mixed))): #
    game_map = df_mixed['game_map'].iloc[index]
    game_type = df_mixed['game_type'].iloc[index]

    winners = list(df_mixed[winner_cols].iloc[index].astype(str))
    losers = list(df_mixed[loser_cols].iloc[index].astype(str))
    #bans =
    dummies_data = dummies_data.append(hero_hot_transform(game_map = game_map, game_type=game_type, winners = winners, losers = losers), ignore_index=True)

In [ ]:
dummies_data.describe()

In [ ]:
print('total mem: ', sum(dummies_data.memory_usage()) / 1000)
print(dummies_data.dtypes)

In [ ]:
dummies_data = pd.concat([dummies_data, df_mixed[['outcome']]], axis = 1)

In [ ]:
dummies_data.tail()

## Prepare train and val sets

In [ ]:
#need this again for new dataframe, cause we attached strings when transforming
#train_cats(dummies_data)

In [ ]:
df_trn, y_trn, nas = proc_df(dummies_data, 'outcome')

In [ ]:
train_required_ratio = 0.8
n_trn = int(len(df_trn) * train_required_ratio)
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
X_train.shape, X_valid.shape

In [ ]:
apply_cats(X_valid, X_train)

Start with simple and stupid

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, size = 20, precision=3)
#?draw_tree

### Try Out Bagging ?
dont remember what is it.. gotta read about it more

In [ ]:
#preds = np.stack([t.predict(X_valid) for t in m.estimators_])
#preds[:,0], np.mean(preds[:,0]), y_valid[0]
#preds.shape

In [ ]:
#plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(10)]);

Subsampling

In [ ]:
#set_rf_samples(2000) #if you set_rf_samples - cannot use OOB score, so dont put the parameter

In [ ]:
#m = RandomForestRegressor(n_estimators=20, n_jobs=-1, oob_score=False)
#%time m.fit(X_train, y_train)
#print_score(m)

In [ ]:
#reset_rf_samples()

In [ ]:
#m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=-1, oob_score=False) #min_samples_leaf = 1-3-5-10-25 are good vals
#%time m.fit(X_train, y_train)
#print_score(m)

things to try:
max_features = 0.5, 'sqrt', 'log2'
leafs = 1, 3, 5, 10, 25, 50, 100...

In [ ]:
#m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, max_features = 0.2, n_jobs=-1, oob_score=True) #max_features = 1-0.5-0.2, sqrt are good vals
#%time m.fit(X_train, y_train)
#print_score(m)

Well seems like max_features < 1 are not helping, also subsampling doesnt help either.  Let's get back to base mode, with more estimators.

In [ ]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

Would feature importance show hero importance in case if I transformed the table to hero-based?

In [ ]:
fi = rf_feat_importance(m, df_trn); fi[:10]
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)
plot_fi(fi[:50]);

In [ ]:
#to_keep = fi[fi.imp>0.005].cols; len(to_keep)
#df_keep = df_trn[to_keep].copy()
#X_train, X_valid = split_vals(df_keep, n_trn)

checking how correlated the features are:

In [ ]:
from scipy.cluster import hierarchy as hc

corr = np.round(scipy.stats.spearmanr(df_trn).correlation, 3)
corr_condensed = hc.distance.squareform(pow((1-corr), 3))
z = hc.linkage(corr_condensed, method='complete')
fig = plt.figure(figsize=(8,16))
dendrogram = hc.dendrogram(z, labels=df_trn.columns, orientation='left', leaf_font_size=10)
plt.show()

ggplot(df, aes())+stat_smooth() is a very powerfull graphics lib, coming from R
use get_sample(df, n) to decrease complexity of graph
https://forums.fast.ai/t/unofficial-lesson-4-classnotes/7569
pdp - partial dependence plot library

since the data is patch dependent, it would be better to weight recent matches more whent training. most modeling algorithms can be adjusted via some weight constant

In [ ]:
from treeinterpreter import treeinterpreter as ti
row = X_valid.values[None,0]; row
df_train, df_valid = split_vals(dummies_data, n_trn)

In [ ]:
#test if tree is properly interpreted
#prediction, bias, contributions = ti.predict(m, row)
#idxs = np.argsort(contributions[0])
#[(a, b, c) for (a, b, c) in zip(dummies_data.columns[idxs], df_valid.iloc[0][idxs], contributions[0][idxs])  if (a in ['game_map', 'game_type'] or b != 0)]

Try interpret a custom data input:

In [ ]:
def prognoses(model, game_map, game_type, winners, losers):
    input_row = np.asarray([hero_hot_transform(game_map = game_map, game_type=game_type, winners = winners, losers=losers)])
    prediction, bias, contributions_input = ti.predict(model, input_row)
    idxs = np.argsort(contributions_input[0])
    interpretations = [(a, b, c) for (a, b, c) in zip(dummies_data.columns[idxs], input_row[0][idxs], contributions_input[0][idxs])  if b != 0]
    rest = [(a, b, c) for (a, b, c) in zip(dummies_data.columns[idxs], input_row[0][idxs], contributions_input[0][idxs])  if b == 0]

    change = sum([c for c in contributions_input[0][idxs]])

    outcome = bias[0] + change
    #print(interpretations)
    #print(bias, change, [c for (a, b, c) in interpretations])
    if bias[0] + change > 0.5:
        print("your team is expected to win (>0.5): ",outcome)
    else:
        print("your team is expected to lose (<0.5): ",outcome)
    return (bias, interpretations, rest)
#this is wrong, because I zip wrong things, so values are right, but titles are not

In [ ]:
#make a function to draw two waterfalls side by side, one for each team
#https://github.com/chrispaulca/waterfall/blob/master/Tree_interpreter_Example.ipynb
#https://github.com/chrispaulca/waterfall
import waterfall_chart

def draw_interpretation_waterfalls(bias, interpretations, rest):
    winner_team_interps = [(a, b, c) for (a, b, c) in interpretations if (a in hero_names and b == 2)]
    loser_team_interps = [(a, b, c) for (a, b, c) in interpretations if (a in hero_names and b == 1)]

    colnames_winner = [a for (a, b, c) in winner_team_interps]
    colnames_loser = [a for (a, b, c) in loser_team_interps]
    conts_winner = [c for (a, b, c) in winner_team_interps]
    conts_loser = [c for (a, b, c) in loser_team_interps]
    
    colname_totals = ['bias','your team', 'opponents','map','type', 'rest']
    map_interp = [c for (a, b, c) in interpretations if (a == 'game_map')]
    type_interp = [c for (a, b, c) in interpretations if (a == 'game_type')]
    rest_interp = [c for (a, b, c) in rest]
    conts_totals = [bias[0],sum(conts_winner),sum(conts_loser), sum(map_interp), sum(type_interp), sum(rest_interp)]

    winner_plot = waterfall_chart.plot(colnames_winner,conts_winner, rotation_value=90, threshold=0.0,formatting='{:,.4f}')
    loser_plot = waterfall_chart.plot(colnames_loser,conts_loser, rotation_value=90, threshold=0.0,formatting='{:,.4f}')
    totals_plot = waterfall_chart.plot(colname_totals,conts_totals, rotation_value=90, threshold=0.0,formatting='{:,.4f}')

#conts = [contributions_input[0][i] for i in range(len(contributions_input[0]))]
#colnames = dummies_data.columns[0:-1].values
#my_plot = waterfall_chart.plot(colnames,conts, rotation_value=90, threshold=0.1,formatting='{:,.3f}')

set inputs:

In [ ]:
map_names

In [ ]:
#print hero names, to copy exactly:
hero_names

In [ ]:
game_map = 'Tomb of the Spider Queen'
game_type = 'UnrankedDraft'
your_team=['Brightwing', "Kael'thas", 'Leoric', "Qhira", 'Malthael']
opponent_tea=['Johanna', 'Malfurion', 'Azmodan', 'Kerrigan', 'Blaze']

calculate and draw out match prognosis:

In [ ]:
bias, prog, rest = prognoses(m, game_map, game_type, your_team, opponent_tea)

In [ ]:
draw_interpretation_waterfalls(bias, prog ,rest)

In [ ]:
input_row = np.asarray([hero_hot_transform(game_map = game_map, game_type=game_type, winners = your_team, losers=opponent_tea)])
prediction = m.predict(input_row)
print(prediction)

# XGBoost part

In [ ]:
import xgboost
my_model_2 = xgboost.XGBRegressor(random_state=0, learning_rate = 0.2, n_estimators = 200)
xgb_weights = pd.Series(np.linspace(0.5, 1, sample_size))

In [ ]:
%time my_model_2.fit(X_train, y_train, sample_weight_eval_set = xgb_weights, verbose=False) # , early_stopping_rounds=5, eval_set=[(X_valid, y_valid)] # throws assersion error
print_score(m)

In [ ]:
?pl

In [ ]:
#import matplotlib.pylab as pl
#plt.rcParams["figure.figsize"] = (10, 15)
#xgboost.plot_importance(my_model_2, height=0.75, importance_type="gain") #, max_num_features=20
#pl.title("xgboost.plot_importance(model)")
#pl.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(my_model_2)
shap_values = explainer.shap_values(df_trn)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar",max_display=X_train.shape[1]) #scrollable
#shap.summary_plot(shap_values[:,5:6], X.iloc[:, 5:6])

In [ ]:
shap.summary_plot(shap_values, df_trn,alpha=0.2)

In [ ]:
?shap.summary_plot

In [ ]:
shap.initjs()

In [ ]:
#to visualize single prediction with XGBoost use: shap:  https://slundberg.github.io/shap/notebooks/Census%20income%20classification%20with%20XGBoost.html
#shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])
input_row = np.asarray([hero_hot_transform(game_map = game_map, game_type=game_type, winners = winners, losers=losers)])
shap.force_plot(explainer.expected_value, shap_values[1000,:], X_train.iloc[0,:])
#How to show the concrete game??? output changes when shap_values[] indexer is changed... but does it input new vals? How to omit zeroes?

In [ ]:
?shap.force_plot

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[:200,:], X_train.iloc[:200,:])

In [ ]:
#this doesnt show anything usefull..
shap.dependence_plot("Auriel", shap_values[10000:], df_trn[10000:], display_features=X_train, x_jitter = 0.15, alpha = 0.2)

In [ ]:
?shap.dependence_plot